In [1]:
import pandas as pd
import numpy as np
import random
import time
import os
import gc

from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LinearRegression

import warnings
warnings.simplefilter('ignore')

In [2]:
df = pd.read_csv('../input/september-folds/train_folds.csv')
df_test = pd.read_csv('../input/tabular-playground-series-sep-2021/test.csv')
sample_solution = pd.read_csv('../input/tabular-playground-series-sep-2021/sample_solution.csv')

df1 = pd.read_csv("../input/september-blending/train_pred_1.csv")
df2 = pd.read_csv("../input/september-blending/train_pred_2.csv")

df_test1 = pd.read_csv("../input/september-blending/test_pred_1.csv")
df_test2 = pd.read_csv("../input/september-blending/test_pred_2.csv")

df = df.merge(df1, on = "id", how = "left")
df = df.merge(df2, on = "id", how = "left")

df_test = df_test.merge(df_test1, on = "id", how = "left")
df_test = df_test.merge(df_test2, on = "id", how = "left")

df.head()

,id,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f113,f114,f115,f116,f117,f118,claim,KFold,pred_1,pred_2
0,0,0.10859,0.004314,-37.566,0.017364,0.28915,-10.25100,135.12,168900.0,3.992400e+14,...,-7.11570,4378.80,1.2096,8.613400e+14,140.1,1.01770,1,0,0.418023,0.413137
1,1,0.10090,0.299610,11822.000,0.276500,0.45970,-0.83733,1721.90,119810.0,3.874100e+15,...,4.14200,913.23,1.2464,7.575100e+15,1861.0,0.28359,0,3,0.324354,0.293616
2,2,0.17803,-0.006980,907.270,0.272140,0.45948,0.17327,2298.00,360650.0,1.224500e+13,...,8.13120,45119.00,1.1764,3.218100e+14,3838.2,0.40690,1,1,0.905333,0.873292
3,3,0.15236,0.007259,780.100,0.025179,0.51947,7.49140,112.51,259490.0,7.781400e+13,...,-16.33600,4952.40,1.1784,4.533000e+12,4889.1,0.51486,1,4,0.543065,0.594655
4,4,0.11623,0.502900,-109.150,0.297910,0.34490,-0.40932,2538.90,65332.0,1.907200e+15,...,-0.43124,3856.50,1.4830,-8.991300e+12,NaN,0.23049,1,3,1.133272,0.956977


In [3]:
useful_features = ["pred_1", "pred_2"]
df_test = df_test[useful_features]

final_predictions = []

scores = []

for fold in range(5):
    xtrain =  df[df.KFold != fold].reset_index(drop = True)
    xvalid = df[df.KFold == fold].reset_index(drop = True)
    xtest = df_test.copy()

    ytrain = xtrain.claim
    yvalid = xvalid.claim
    
    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]
    
    model = LinearRegression()
    model.fit(xtrain, ytrain)
    
    preds_valid = model.predict(xvalid)
    test_preds = model.predict(xtest)
    final_predictions.append(test_preds)
    auc = roc_auc_score(yvalid, preds_valid)
    print(fold, auc)
    scores.append(auc)

print(np.mean(scores), np.std(scores))

0 0.7999154356637097
1 0.8015487342506012
2 0.800633837417282
3 0.8020683149481767
4 0.8017085403854305
0.80117497253304 0.0007876183700257251


In [4]:
sample_solution.claim = np.mean(np.column_stack(final_predictions), axis = 1)
sample_solution.to_csv("submission.csv", index = False)